https://github.com/arthurdouillard/keras-effnet

In [6]:
from keras.models import Model
from keras.layers import *
from keras.activations import *
from keras.callbacks import *


def get_post(x_in):
    x = LeakyReLU()(x_in)
    x = BatchNormalization()(x)
    return x

def get_block(x_in, ch_in, ch_out):
    x = Conv2D(ch_in,
               kernel_size=(1, 1),
               padding='same',
               use_bias=False, data_format='channels_first')(x_in)
    x = get_post(x)

    x = DepthwiseConv2D(kernel_size=(1, 3), padding='same', use_bias=False, data_format='channels_first')(x)
    x = get_post(x)
    x = MaxPool2D(pool_size=(2, 1),
                  strides=(2, 1), data_format='channels_first')(x) # Separable pooling

    x = DepthwiseConv2D(kernel_size=(3, 1),
                        padding='same',
                        use_bias=False, data_format='channels_first')(x)
    x = get_post(x)

    x = Conv2D(ch_out,
               kernel_size=(2, 1),
               strides=(1, 2),
               padding='same',
               use_bias=False, data_format='channels_first')(x)
    x = get_post(x)

    return x


def Effnet(input_shape, nb_classes, include_top=True, weights=None):
    x_in = Input(shape=input_shape)

    x = get_block(x_in, 32, 64)
    x = get_block(x, 64, 128)
    x = get_block(x, 128, 256)

    if include_top:
        x = Flatten()(x)
        x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=x_in, outputs=x)

    if weights is not None:
        model.load_weights(weights, by_name=True)

    return model

In [7]:
model = Effnet((1,64,321),2)

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1, 64, 321)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 64, 321)       32        
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 32, 64, 321)       0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 32, 64, 321)       1284      
_________________________________________________________________
depthwise_conv2d_2 (Depthwis (None, 32, 64, 321)       96        
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 32, 64, 321)       0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 32, 64, 321)       1284      
__________